In [1]:
# Install dependencies
import random
import pandas as pd
import itertools
from tile import Tile
from plate import Plate
from typing import List

In [2]:
yellow = "yellow"
red = "red"
blue = "blue"
green = "green"
black = "black"
# Test version of Board class
class Space():
    """Space on score board"""

    def __init__(self, color: str = None, minus_value: int = None):
        self.tile = None
        self.color = color
        self.minus_value = None

    @property
    def _tile(self, _tile: str):
        self.tile = _tile

    @property
    def _minus_value(self, _minus_value: int):
        self.minus_value = _minus_value
        
class Board():
    """5x5 Playing board to track game progress"""

    def __init__(self, name: str):
        self.game_over = False
        self.name = name
        self.spaces = [
            [Space(yellow), Space(blue), Space(green), Space(red), Space(black)],
            [Space(black), Space(yellow), Space(blue), Space(green), Space(red)],
            [Space(red), Space(black), Space(yellow), Space(blue), Space(green)],
            [Space(green), Space(red), Space(black), Space(yellow), Space(blue)],
            [Space(blue), Space(green), Space(red), Space(black), Space(yellow)],
            ]

    def check_rows(self):
        """
        Check if the end-game condition has been met. Game ends if
        any row on a Board is filled with tiles
        """
        for i in range(len(self.spaces)):
            has_tiles = [self.spaces[i][j].tile for j in range(len(self.spaces))]
            if all(has_tiles):
                self.game_over = True
        return

class Factory():
    """
    Board where tiles are collected by players before being moved
    to the scoring board.
    """

    def __init__(self):
        self.floor = Floor()
        self.rows = {
            "row_1": {
                "red": 1,
                "green": 1,
                "blue": 1, 
                "yellow": 1, 
                "black": 1,
            },
            "row_2": {
                "red": 2,
                "green": 2,
                "blue": 2, 
                "yellow": 2, 
                "black": 2,
            },
            "row_3": {
                "red": 3,
                "green": 3,
                "blue": 3, 
                "yellow": 3, 
                "black": 3,
            },
            "row_4": {
                "red": 4,
                "green": 4,
                "blue": 4, 
                "yellow": 4, 
                "black": 4,
            },
            "row_5": {
                "red": 5,
                "green": 5,
                "blue": 5, 
                "yellow": 5, 
                "black": 5,
            },
            "row_minus": {
                "red": 7,
                "green": 7, 
                "blue": 7, 
                "yellow": 7,
                "black": 7,
            },
            "row_beyond_minus": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0,
                "black": 0,
            },
        }
        self.playable_options = self.get_playable_options()

    def get_playable_options(self):
        """Convert self.rows to a pandas dataframe of all playable options"""
        options_df = pd.DataFrame(self.rows)
        options_df = options_df.reset_index()
        options_df = pd.melt(
            options_df, 
            id_vars=["index"], 
            value_vars=[x for x in options_df if x.startswith("row")]
            ).rename(columns={"index": "color", "value":"playable_spaces"})
        return options_df
            

class Floor():
    """
    Minus section of a player's Factory
    """

    def __init__(self):
        self.spaces = [
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=2),
            Space(minus_value=2),
            Space(minus_value=3),
            Space(minus_value=3),
            ]
        self._discard = []

    @property
    def discard(self):
        return self._discard

    @discard.setter
    def discard(self, val):
        self._discard = val
        
    def reset(self):
        """Reset Floor to original state"""
        for space in self.spaces:
            if space.tile:
                self.discard.append(space.tile)

        self.spaces = [
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=1), 
            Space(minus_value=2),
            Space(minus_value=2),
            Space(minus_value=3),
            Space(minus_value=3),
            ]

class Player():
    """
    Represent a player in the game
    """

    def __init__(self, name=str):
        self.name = name
        self.factory = Factory()
        self.board = Board("player_1")


class Table():
    """ 
    Represents where left over tiles from plates go
    """
    
    def __init__(self, n_plates: int = 5, players=[Player]):
        colors = [red, green, blue, yellow, black]
        self._players = players
        self.n_plates = n_plates
        self.bag = [Tile(color) for color in colors for _ in range(20)]
        self.plates = {
            "plate_1": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_2": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_3": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_4": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_5": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_leftover": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
        }
    
    @property
    def players(self):
        return self._players
    
    @players.setter
    def players(self, updated_players: List[Player]):
        self._players = updated_players

    @staticmethod
    def get_discard_tiles(players):
        """Combine discard tiles from all players"""
        discard_tiles = []
        for player in players:
            discard_tiles.extend(player.factory.floor.discard)

    def fill_plates(self, discard_tiles: List[Tile]):
        """
        Fill plates with tiles from bag

        :param discard: Player's discard piles from their floors
        """
        self.empty_plates()
        n_tiles = self.n_plates * 5

        # Check if there are enough plates in the bag
        if len(self.bag) >= n_tiles:
            self.deal_plates()
        # Re-fill bag with tiles from players' minus section
        else:
            self.bag.extend(discard_tiles)
            self.deal_plates()

            # Set discard piles back to being empty for each player
            for player in self.players:
                player.factory.floor.discard = []

    def deal_plates(self):
        """Shuffle bag and place 4 random tiles on each plate"""
        # Shuffle tiles
        plate_names = [name for name in self.plates if name[-1].isdigit()]
        random.shuffle(self.bag)
        for plate in plate_names:
            for _ in range(4):
                tile = self.bag.pop()
                self.plates[plate][tile.color] += 1


    def empty_plates(self):
        """Reset all plates to an empty state"""
        self.plates = {
            "plate_1": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_2": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_3": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_4": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_5": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
            "plate_leftover": {
                "red": 0,
                "green": 0, 
                "blue": 0, 
                "yellow": 0, 
                "black": 0, 
            },
        }

    @staticmethod
    def get_playable_options(plates):
        """Convert self.rows to a pandas dataframe of all playable options"""
        options_df = pd.DataFrame(plates)
        options_df = options_df.reset_index()
        options_df = pd.melt(
            options_df, 
            id_vars=["index"], 
            value_vars=[x for x in options_df if x.startswith("plate")]
            ).rename(columns={"index": "color", "value":"playable_tiles"})
        return options_df

def game_state(table: Table):
    """Print out current game state"""
    player_1 = table.players[0]
    player_2 = table.players[1]
    print(f"Current players: {len(table.players)}")
    print(f"Player_1 factory: {player_1.factory.playable_options}")
    print(f"Player_2 factory: {player_2.factory.playable_options}")
    print()
    print(f"Player_1 discard: {player_1.factory.floor.discard}")
    print(f"Player_2 discard: {player_2.factory.floor.discard}")
    print()
    print(f"Current plate setup:")
    for plate in table.plates:
        print(f"{plate}: {table.plates[plate].items()}")
    print()



In [21]:
# Set up game with 2 players
player_1 = Player("Kaci")
player_2 = Player("Avi")
table = Table(n_plates=5, players=[player_1, player_2])
table.fill_plates(player_1.factory.floor.discard + player_2.factory.floor.discard)

# Playable options of plates on table
table_options = table.get_playable_options(table.plates)
table_options = table_options.where(table_options["playable_tiles"] != 0).dropna()
print("Table options:")
print(table_options.index.values)
print()
# Playable options of player factories
player_options = player_1.factory.playable_options
player_options = player_options.where(player_options["playable_spaces"] != 0).dropna()
print("Player factory options:")
print(player_options.index.values)


# Get all possible options and randomly select one
all_options = list(itertools.product(table_options.index.values, player_options.index.values))
random.choice(all_options)

Table options:
[ 0  1  2  3  5  7  9 10 11 14 15 18 19 20 21 22 23]

Player factory options:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]


(19, 12)